In [1]:
using PCT

Precompiling PCT
  ✓ PCT
  1 dependency successfully precompiled in 2 seconds. 7 already precompiled.


## $x^{\dagger} A x$

In [3]:
f, _ = @pct (A::Her) -> 𝒫((x::CV) -> sum((i, j), x(i)' * A(i, j) * x(j)))
blaserize(f)

(A) -> 
    𝒫((x) -> 
        x⁺⋅A⋅x)

In [20]:
df = redux(vdiff(f), settings=symmetry_settings) |> blaserize

(A) -> 
    (x) -> 
        2.0⋅A⋅x

## Trace of Product

In [26]:
f, _ = @pct begin
    tr = (X::CM) -> sum(i, X(i, i))
    matmul = (A::CM, B::CM) -> (i::N, j::N) -> sum(n, A(i, n) * B(n, j))
    pullback((A::CM, B::CM) -> tr(matmul(A, B) + matmul(B, A)))
end; f

let 
    tr = (X) -> 
        ∑((i), X(i, i))
    matmul = (A, B) -> 
        (i, j) -> 
            ∑((n), (A(i, n)⋅B(n, j)))
    𝒫((A, B) -> 
        tr((matmul(A, B)+matmul(B, A))))
end

In [37]:
redux(vdiff(eval_all(f)); settings=symmetry_settings) |> blaserize

(A, B) -> 
    2.0⋅B⁺, 2.0⋅A⁺

## Conjugate Gradient

In [127]:
f, _ = @pct (A::Sym, r::RV, p::RV, b::RV, x::RV) -> begin
    E = (x::RV) -> sum((i, j), 0.5 * x(i) * A(i, j) * x(j)) + sum(i, x(i) * b(i))
    pullback((α::R, β::R) -> E((i::N) -> x(i) + α * (r(i) + β * p(i))))
end; blaserize(f)

(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (xᵀ⋅b+(0.5⋅xᵀ⋅A⋅x))
        𝒫((α, β) -> 
            E((x+α⋅(r+β⋅p))))
    end

In [126]:
df = vdiff(f) |> blaserize

(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (xᵀ⋅b+(0.5⋅xᵀ⋅A⋅x))
        (α, β) -> 
            𝒫(E)((x+α⋅(r+β⋅p)), 1)ᵀ⋅(r+β⋅p), (α⋅𝒫(E)((x+α⋅(r+β⋅p)), 1)ᵀ⋅p)
    end

In [52]:
df = redux(vdiff(eval_all(f)); settings=symmetry_settings) |> blaserize

(A, r, p, b, x) -> 
    (α, β) -> 
        ((r+β⋅p)ᵀ⋅b+(x+α⋅(r+β⋅p))ᵀ⋅A⋅(r+β⋅p)), (α⋅(bᵀ⋅p+pᵀ⋅A⋅(x+α⋅(r+β⋅p))))

## Lagrangian Mechanics

In [53]:
_, ctx = @pct begin
    @space X begin
        type = (RF, ) -> RF
        linear = true
    end
end; 

In [54]:
f, _ = @pct _ ctx (L::RO, D::X) -> pullback((r::RF) -> sum((t::R), L(r(t), D(r)(t)))); f

(L, D) -> 
    𝒫((r) -> 
        ∑((t), L(r(t), D(r)(t))))

In [55]:
redux(vdiff(f); settings=custom_settings(:clench_sum=>true))

(L, D) -> 
    (r) -> 
        ((_d) -> 
            𝒫((_z_2) -> 
                L(_z_2, D(r)(_d)))(r(_d), 1)+D'((_d) -> 
            𝒫((_z_1) -> 
                L(r(_d), _z_1))(D(r)(_d), 1)))

## Hartree Fock Gradient and Hessian Vector Product

In [128]:
_, ctx = @pct begin    
    @space T begin
        type = (I, I, I, I) -> C
        symmetries = (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))
    end
end;

In [130]:
f, _ = @pct _ ctx (J::T) -> pullback((C::CM) -> sum((i, j, p, q, r, s),
    C(p, i)' * C(q, i) * C(r, j)' * C(s, j) * J(p, q, r, s))); f

(J) -> 
    𝒫((C) -> 
        ∑((i, j, p, q, r, s), (C(p, i)'⋅C(r, j)'⋅J(p, q, r, s)⋅C(s, j)⋅C(q, i))))

In [58]:
df = redux(vdiff(f); settings=symmetry_settings)

(J) -> 
    (C) -> 
        (_d, _d_1) -> 
            (∑((j, p, r, s), (C(s, j)'⋅J(_d, p, s, r)⋅C(p, _d_1)⋅C(r, j)))⋅4.0)

In [59]:
df |> get_body |> decompose |> pp |> simplify |> first

(C, _k) -> 
    (_a_2, _a_3) -> 
        (∑((_d, _d_1, p, s), (J(_d, p, s, _a_2)'⋅C(p, _d_1)'⋅4.0⋅_k(_d, _d_1)⋅C(s, _a_3)))+∑((_d, _d_1, p, r), (_k(_d, _d_1)'⋅4.0⋅J(_d, p, _a_2, r)⋅C(p, _d_1)⋅C(r, _a_3)))+∑((_d, j, r, s), (J(_d, _a_2, s, r)'⋅C(r, j)'⋅4.0⋅_k(_d, _a_3)⋅C(s, j))))

## Foster Boys

In [131]:
f, _ = @pct (R::Her, R2::Her) -> pullback((C::CM) -> begin 
        r = (i::N) -> sum((p, q), R(p, q) * C(p, i)' * C(q, i))
        sum(i, sum((p, q), R2(p, q) * C(p, i)' * C(q, i))) - sum(i, r(i)' * r(i))
    end); blaserize(f)

(R, R2) -> 
    𝒫((C) -> 
        let 
            r = (i) -> 
                (C⁺⋅R⋅C)(i, i)
            ((-1.0⋅r⁺⋅r)+tr(C⋅C⁺⋅R2))
        end)

In [135]:
df = redux(vdiff(eval_all(f)); settings=symmetry_settings) |> blaserize

LoadError: MethodError: no method matching name(::PCTVector)

[0mClosest candidates are:
[0m  name([91m::Domain[39m)
[0m[90m   @[39m [35mPCT[39m [90m~/kaer_morhen/projects/QLD/PCT/src/[39m[90m[4mtype.jl:62[24m[39m
[0m  name([91m::Var[39m)
[0m[90m   @[39m [35mPCT[39m [90m~/kaer_morhen/projects/QLD/PCT/src/[39m[90m[4mnodes.jl:94[24m[39m


## Localized Wannier Functions

In [62]:
_, ctx = @pct begin
    @domain BZ begin
        base = I
        periodic = true
    end
    
    @domain X begin
        symmetric = true
        contractable = false
    end
    
    @space Mmn begin
        type = (N, N, BZ, BZ) -> C
        symmetries = (((2, 1, 4, 3), :conj),)
    end

    @space SV begin
        type = (I,) -> C
        symmetries = (((1,), :ineg),)
    end
    
    @space Gauge begin
        type = (N, N, BZ) -> C
    end
end;

In [63]:
f, _ = @pct _ ctx (S::Mmn, w::SV) -> pullback((U::Gauge) -> 
    begin
        ρ = (n::N, b::X) -> sum((k::BZ, p, q), U(p, n, k)' * S(p, q, k, k + b) * U(q, n, k + b))
        sum((n, b::X), ρ(n, b)' * ρ(n, b))
    end); f

(S, w) -> 
    𝒫((U) -> 
        let 
            ρ = (n, b) -> 
                ∑((k, p, q), (U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))
            ∑((n, b), (ρ(n, b)'⋅ρ(n, b)))
        end)

In [64]:
# df = vdiff(eval_all(f); settings=custom_settings(:logging=>true; preset = symmetry_settings))
df = redux(vdiff(eval_all(f)); settings=symmetry_settings)

(S, w) -> 
    (U) -> 
        (_d, _d_1, _d_2) -> 
            (∑((b, p, k, _p, q), (U(_p, _d_1, (b+k))'⋅U(q, _d_1, k)⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b))⋅S(_p, q, (b+k), k)))⋅4.0)

In [65]:
_, ctx = @pct begin
    @space Tau begin
        type=(R, RV) -> RM
    end
    
    @space Nu begin
        type=(RV, ) -> R
    end
        
    @space S begin
        type=(RV, )->RV
    end
end;

In [66]:
f, _ = @pct _ ctx (τ::Tau, ν::Nu, p::S) -> :vdiff(pullback((x::RV) -> τ(ν(x), p(x)))); f

(τ, ν, p) -> 
    vdiff̂(𝒫((x) -> 
        τ(ν(x), p(x))))

In [15]:
df = process_directive(f)

(τ, ν, p) -> 
    (x) -> 
        (𝒫(ν)(x, 𝒫((_z) -> 
            τ(_z, p(x)))(ν(x), 1))+𝒫(p)(x, 𝒫((_z_1) -> 
            τ(ν(x), _z_1))(p(x), 1)))